<a href="https://colab.research.google.com/github/jepennerhahn/EXAFS-simulations/blob/main/InteractiveEXAFSPlots_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<center><img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQrnKPDk-4bRvj-erHLNtLwrNPjqz8hglreMjrtz4sNikcPvE4C3EN_Eei742lFWnh2Yw&usqp=CAU">
<h1><font size="+3">EXAFS Simulator</font></h1>
<h2>
Biophysics 401
</font> 
</h2>
</center>
This provides an approximation of the EXAFS for 2 shells of scatterers using the McKale amplitude and phase factors for the scatterer and the Teo & Lee parameters for the central atom phase shift.

In [1]:
#@title Initialization
%%capture
'''Copyright (c) 2022 J.E. Penner-Hahn

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:
The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.'''

import numpy as np
import scipy
import ipywidgets as widgets
import matplotlib.pyplot as plt
import pickle
import os

#Read the necessary amplitude and phase parameters from github - these are
#np.interp2d interpolators.  elemDict is a dictionary of symbol to Z
#NOTE:  Interpolators are only defined for k=2-20 and for Z=5-85.  
#Values outside this range will be wrong.
#Usage:  amplitude = amp(k,Z)

if not os.path.isdir('EXAFS-simulations/'):
    !git clone "https://github.com/jepennerhahn/EXAFS-simulations.git"
with open('EXAFS-simulations/elem.pk', 'rb') as file_in:
    elemDict = pickle.load(file_in)
with open('EXAFS-simulations/McKale.pk', 'rb') as file_in:
    [amp,phs] = pickle.load(file_in)
with open('EXAFS-simulations/TeoLee.pk', 'rb') as file_in:
    central = pickle.load(file_in)

#Define a range for k for calculation. Go to kmax = 100 to avoid aliasing in FFT
kmin = 0.
kmax = 100.
kstep = 0.1
k = np.arange(kmin,kmax+kstep/2,kstep)

# def gauss(c,w,x):
#     return np.exp(-(x-c)**2/(2*w**2))/w/np.sqrt(2*np.pi)
def exafs(N,R,Amp,Phs,sigma,k):
    '''Function to calculate exafs.  Assumes that amplitude and phase will be arrays of the
    same length as k.
    The negative sign at the start reflects a (-1)^l term in exafs expression.
    With this, the imaginary part of the FT peaks at magnitude peak, when phase == 0 throughout'''
    return -Amp*(N/k/R**2)*np.exp(-2*sigma*k**2)*np.sin(2*k*R+Phs)
def kWindow(kLow,kHigh,kw,k):
    leftSide = (1+np.exp(-(k-kLow)/kw))**-1
    rightSide = 1.-(1+np.exp(-(k-kHigh)/kw))**-1
    return leftSide*rightSide
def plotExafs(R,N,sig,elem,FTrange,remPhase):
    '''Plot the exafs for some number of scatterers.  For each scatterer, 
    the distance, coordination number, Debye-Waller factor, and atomic number are 
    passed in R, N, sig, and elem


    The FTrange to be plotted is passed in range as kmin, kmax, and window

    incPhase is a boolean value indicating whether the phase should be included or not
    
    This assumes that k has been defined, that amp and phs are interpolators to 
    give McKale scattering amplitude and phase, and that elemDict has been defined'''

    NS = len(R) #Number of shells
    y = np.zeros((len(k),NS))
    for n in range(NS):
        Z = elemDict[elem[n]]
# If the users has not set the remove phase checkbox, add in the scatterer phase and the
# central atom phase. Currently configured to have Z=29 for the central atom.
# This is not a user adjustable parameter

#Note: first point is omitted to avoid divide by zero error.

        if not remPhase:
          phase = phs(k[1:],Z) + central(k[1:],29)
        else:
          phase = np.zeros(len(k)-1)
        y[1:,n] += exafs(N[n],R[n],amp(k[1:],Z), phase,sig[n],k[1:])*k[1:]**3
        # y[:,n] += exafs(N[n],R[n],amp(k,Z), phs(k,Z),sig[n],k)*k**3
    yTot = np.sum(y, axis = 1)

    #Apply a filter window
    kLow,kHigh,kw = FTrange
    window = kWindow(kLow,kHigh,kw,k)
    yFilt = yTot*window

    FT = np.fft.fft(yFilt)/float(k.size)
    FTparts = np.zeros((len(FT),NS))
    for n in range(NS):
        FTparts[:,n] = abs(np.fft.fft(y[:,n]*window))/float(k.size)
    R = np.fft.fftfreq(k.size, d = kstep)*np.pi

    fig,ax = plt.subplots(nrows = 1, ncols = 2, figsize = (15,5))
    for n in range(NS):
        ax[0].plot(k,y[:,n], label = 'Shell {:1d}: {:.1f} {}'.format(n+1,N[n],elem[n]))
    ax[0].plot(k,yTot, linewidth = 2, label = 'Sum', color = 'black')
    # ax[0].plot(k,yFilt)
    ax[0].plot(k,window*np.max(yTot), color = 'grey', linestyle = 'dashed')
    ax[0].set_xlim((0,kHigh+2*kw))
    ax[0].set_xlabel('$k(\AA^{-1}$)', fontsize = 14)
    ax[0].set_ylabel('$EXAFS \cdot k^3$', fontsize = 14)
    ax[0].legend()
    for n in range(NS):
        ax[1].plot(R,FTparts[:,n], label = 'Shell {:1d}: {:.1f} {}'.format(n+1,N[n],elem[n]))
    ax[1].plot(R,FT.imag, color = 'grey', linestyle = 'dashed', label = 'Imaginary')
    # ax[1].plot(R,FT.real, label = 'Real')
    ax[1].plot(R,abs(FT), 'k', label = 'Magnitude')
    ax[1].set_xlim((0,6))
    ax[1].grid()
    ax[1].legend()
    ax[1].set_xlabel('$R(\AA)$)', fontsize = 14)
    ax[1].set_ylabel('FT magnitude', fontsize = 14)
    plt.show()
    return

def interactEXAFS(R1,R2,N1,N2,sig1,sig2,e1,e2,k1,k2,k3,p1):
    plotExafs([R1,R2],[N1,N2],[sig1/1000.,sig2/1000.],[e1,e2],[k1,k2,k3],p1)

#Define the sliders that we'll need
p1 = widgets.Checkbox(value = False, description = 'Remove phase')
k1 = widgets.FloatSlider(description = 'k_min', min = 1., max = 5, step = 0.01, value = 2.)
k2 = widgets.FloatSlider(description = 'k_max', min = 10, max = 20, step = 0.01, value = 12.)
k3 = widgets.FloatSlider(description = 'k_window', min = .02, max = 2, step = 0.01, value = .25)
e1 = widgets.Dropdown(options = ['N','O','S','Cl','Cu', 'Au'], value = 'N', description = 'Scatt.1')
e2 = widgets.Dropdown(options = ['N','O','S','Cl','Cu', 'Au'], value = 'N', description = 'Scatt.2')
R1 = widgets.FloatSlider(description = 'R', min = 1.6, max = 3.5, step = .01, value = 2.)
R2 = widgets.FloatSlider(description = 'R', min = 1.6, max = 3.5, step = .01, value = 2.)
N1 = widgets.FloatSlider(description = 'N', min = 0, max = 6, step = .1, value = 4)
N2 = widgets.FloatSlider(description = 'N', min = 0, max = 6, step = .1, value = 0)
sig1 = widgets.FloatSlider(description = 'sig^2 x 10^3', min = 0.1, max = 8, step = .1, value = 3)
sig2 = widgets.FloatSlider(description = 'sig^2 x 10^3', min = 0.1, max = 8, step = .1, value = 3)
ui = widgets.VBox([widgets.HBox([p1,k1,k2,k3]),widgets.HBox([e1,R1, N1, sig1]),
                   widgets.HBox([e2, R2, N2, sig2])])


In [2]:
#@title Enter your parameters.  The initial setting is a single shell of 4 N at 2 A. { run: "auto" }

out = widgets.interactive_output(interactEXAFS, 
                           {'R1': R1, 'R2': R2, 'N1': N1, 'N2': N2, 'sig1': sig1, 'sig2': sig2,
                            'e1': e1, 'e2': e2, 'k1': k1, 'k2': k2, 'k3': k3, 'p1': p1})
display(ui,out)

Output()